# <h1>Homework 3

# <h1> Problem 1

For a binary mixture with two adjustable parameters $A_{12}$ and $A_{21}$ equilibrium relations are <br/>
$p = x_1\;exp(A_{12}(\frac{A_{21}x_2}{A_{12}x_1 + A_{21}x_2})^2)p_{water}^{sat} + x_2\;exp(A_{21}(\frac{A_{12}x_1}{A_{12}x_1 + A_{21}x_2})^2)p_{1,4\;dioxane}^{sat}$
<br/> The saturation pressure is given as,
<br/> $log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3}$
<br/> We have also been given that,
<br/> $x_1 + x_2 = 1$
<br/> A table of measures for $x_1$ and corresponding $p$ is also given
<br/> We need to solve this problem like a least squares problem

# <h2> Part 1

Least Squares formulation
<br/> $min_{A_{12},A_{21}} \sum (p - p(A_{12},A_{21}))^{2}$
<br/> This means that we need to minimize the sum of squared difference all actual $p$ and calculated $p$ by modifying $As$.

#<h2> Part 2

In order to get a solution as close as possible to the optimum solution, we cannot linearize the given model.
<br/> To solve the problem with its non-linearity intact we solve the problem using gradient descent.

In [131]:
def p_sat(a1,a2,a3):
    return 10**(a1 - ((a2)/(a3+20)))

In [132]:
p_water = p_sat(8.07131,1730.63,233.426)
p_diox = p_sat(7.43155,1554.679,240.337)

In [133]:
import torch as t
from torch.autograd import Variable
import math
import numpy as np

In [134]:
def sat(x1,x2,p_water,p_diox,x):
    return x1*t.exp(x[0]*((x[1]*x2)/(x[0]*x1 + x[1]*x2))**2)*p_water + x2*t.exp(x[1]*((x[0]*x1)/(x[0]*x1 + x[1]*x2))**2)*p_diox

In [135]:
x1 = t.tensor(np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]))
x2 = 1-x1

In [136]:
x = Variable(t.tensor([1.0,0.5]),requires_grad = True)
# = x.detach().numpy()

In [137]:
p = t.tensor([28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5])

In [138]:
def square_loss(act,pred):
    return t.sum((act-pred)**2)

In [139]:
a = 0.01
for i in range(1000):
    p_pred = sat(x1,x2,p_water,p_diox,x)
    loss = square_loss(p,p_pred)
    loss.backward()
    with t.no_grad():
        x -=a*x.grad
        x.grad.zero_()

In [140]:
print(x.data.numpy())
print(loss.data.numpy())

[-2219.3127 -1437.3357]
11028.58605650331
